# Mustaeen Ahmed: Assignment 8

In [1]:
import pygame
import moderngl
from pyglm import glm
from loadModelUsingAssimp_V2 import create3DAssimpObject

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()

WIDTH = 840
HEIGHT = 480

clock = pygame.time.Clock()
running = True

pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_FORWARD_COMPATIBLE_FLAG, True)

In [3]:
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.set_mode((WIDTH, HEIGHT), flags=pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 08: Mustaeen Ahmed")
gl = moderngl.get_context()
gl.info['GL_VERSION']

'4.1 Metal - 89.4'

In [4]:
assimp_object = create3DAssimpObject("chair_table_class/scene.gltf", verbose=False, textureFlag=True, normalFlag=True)

Accumulates Normals in the geomDataList
Accumulates Textures in the geomDataList


In [5]:
bound = assimp_object.bound

displacement_vector = 2 * bound.radius * glm.vec3(0, 0, 1)

target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0, 1, 0)

fov_radian = glm.radians(45)
aspect = WIDTH / HEIGHT
near = bound.radius
far = 3 * bound.radius
perspective_matrix = glm.perspective(fov_radian, aspect, near, far)

In [6]:
vertex_shader_code = '''
    #version 330 core
    
    layout (location=0) in vec3 position;
    layout (location=1) in vec2 uv;
    layout (location=2) in vec3 normal;

    uniform mat4 model, view, perspective;
    out vec2 f_uv;
    out vec3 f_normal;

    void main() {
        f_uv = uv;
        vec4 P = perspective * view * model * vec4(position, 1);
        gl_Position = P;
        mat3 normalMatrix = mat3(transpose(inverse(model)));
        f_normal = normalize(normalMatrix * normal);
    }
'''

fragment_shader_code = '''
    #version 330 core

    in vec2 f_uv;
    in vec3 f_normal;
    uniform sampler2D map;
    uniform vec3 light;

    out vec4 out_color;

    void main() {
        vec3 materialColor = texture(map, f_uv).rgb;
        vec3 N = normalize(f_normal);
        vec3 L = normalize(light);
        vec3 color = materialColor * clamp(dot(N, L), 0, 1);
        out_color = vec4(color, 1);
    }
'''

In [7]:
program = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)
f_mat = "3f 3f 2f"
variables = ["position", "normal", "uv"]
renderables = assimp_object.getRenderables(gl, program, f_mat, variables)

samplers = assimp_object.getSamplers(gl)

In [ ]:
alpha = 0
pause = True
gl.enable(gl.DEPTH_TEST)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type == pygame.KEYDOWN):
            if event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspective_matrix = glm.perspective(fov_radian, width / height, near, far)

    new_displacement_vector = glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0, 1, 0))
    eye_point = target_point + new_displacement_vector
    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.5, 0.5, 0.0)

    program['view'].write(viewMatrix)
    program['perspective'].write(perspective_matrix)
    program['light'].write(new_displacement_vector)
    assimp_object.render(program, renderables, samplers, M = glm.mat4(1))

    pygame.display.flip()
    clock.tick(60)
    if not pause:
        alpha = alpha + 1
        if alpha > 360:
            alpha = 0

pygame.display.quit()